##### Import libs

In [1]:
import pandas as pd
from decouple import config
import ast
from collections import defaultdict
import numpy as np
# Load enviroment variables.
ROOT_PATH       = config('ROOT_PATH')
DATA_RAW_PATH   = config('DATA_RAW_PATH')

##### Load data

In [51]:
raw_data = pd.read_csv(ROOT_PATH+DATA_RAW_PATH+'dataframe_raw_properties.csv',low_memory=False)
data_ids = pd.read_csv(ROOT_PATH+DATA_RAW_PATH+'dataframe_ids.csv',low_memory=False)

In [34]:
def transform_dict_features_in_columns (dataframe:pd.DataFrame, column_to_transform:str) -> pd.DataFrame:
    '''
        This method transform a string in a list of dict format in a dataframe, when
        each element will be transformed in a column of dataframe.

        Parameters :
            dataframe : A dataframe to looking for
            column_to_transform : A column that contains data to transform
    '''
    dict_ = defaultdict(list)
    for index in range(len(dataframe)) :
        line_column_to_transform = ast.literal_eval(dataframe[column_to_transform][index].strip('"'))
        if len(line_column_to_transform) > 0 :
            for elem in line_column_to_transform:
                dict_[elem['key']].append(elem['value'])
        else : 
            for key in dict_ :
                dict_[key].append(np.nan)
                
    return pd.DataFrame.from_dict(dict_)

##### Process data

In [44]:
# Create dataframe with data
data_properties = transform_dict_features_in_columns(raw_data,'amenities')
# Concatenate data
data_tmp = pd.concat([data_ids,data_properties],axis=1)
# Drop NaN Lines
data_tmp = data_tmp.dropna(axis=0)
# Export to csv
data_tmp.to_csv(ROOT_PATH+DATA_RAW_PATH+'dataframe_merged.csv')